In [1]:
from data_generators import get_iterator, get_dataset
from classifiers import theme_classifier

In [2]:
import torch
import torch.nn as nn

In [3]:
from torchtext.vocab import GloVe 
GLOVE_EMBEDDING = GloVe(name="6B", dim=300)

In [4]:
train_dataset, val_dataset, test_dataset, review_text_FIELD, theme_FIELD = get_dataset(vectors = 
                                                                                       GLOVE_EMBEDDING)

In [5]:
batch_size = 32
train_iter = get_iterator(train_dataset, batch_size, train=True, shuffle=False, repeat=False)
val_iter = get_iterator(val_dataset, batch_size, train=False, shuffle=False, repeat=False)
test_iter = get_iterator(test_dataset, batch_size, train=False, shuffle=False, repeat=False)

In [6]:
list_train = list(val_iter)

In [7]:
batch = list_train[600]
x = batch.review_text.transpose(1, 0).int()[:10]
y = batch.theme.int()

for idx in range(x.shape[0]):
    #print(x.shape, y.shape)
    print("{} | {}".format(' '.join([train_dataset.fields['review_text'].vocab.itos[_] for _ in x[idx]]),
         train_dataset.fields['theme'].vocab.itos[y[idx]]))

as usual sad true story ... but great acting | plot
anything with jennifer lawrence is worth your time . | other
you 'll find them in `` dirty grandpa . | other
no direction , worthwhile plot , or acting . | plot
although be warned there are some gory scenes . | plot
one of the best musical movie for me . | other
so disappointed in jake gyllenhall for this one . | other
it takes much more than a balance to walk | other
that highlights the greatest of a true thriller . | other
overall , though , it left me cold . | other


In [8]:
#review_text_FIELD.vocab.vectors.shape, len(review_text_FIELD.vocab.itos)

In [9]:
theme_FIELD.vocab.itos

['<unk>', 'other', 'plot', 'acting', 'effect', 'production']

In [10]:
vocab_size = review_text_FIELD.vocab.vectors.shape[0]
label_size = len(theme_FIELD.vocab) - 1
emb_dim = review_text_FIELD.vocab.vectors.shape[1]
vectors = train_dataset.fields["review_text"].vocab.vectors
hidden_dim = 500
layers = 2
dropout = .2

label_size

5

In [11]:
class BaseModel(nn.Module):

    def __init__(self, 
                 ninp = vocab_size, 
                 linp = label_size, 
                 emb_dim = emb_dim, 
                 emb_lab = 20,
                 nhid = hidden_dim, 
                 nout = vocab_size, 
                 nlayers = layers, 
                 dropout = dropout, 
                 vectors = vectors,
                 pretrained = False):
        super().__init__()
        
        self.ninp = ninp
        self.linp = linp
        self.emb_dim = emb_dim
        self.emb_lab = emb_lab
        self.nhid = nhid
        self.nout = nout
        self.nlayers = nlayers
        self.drop = nn.Dropout(dropout)

        self.word_embedding = nn.Embedding(ninp, emb_dim)
        self.label_embedding = nn.Embedding(linp, emb_lab)
        
        self.rnn = nn.LSTM(emb_dim + emb_lab, nhid, nlayers, dropout=dropout)
        self.rnn.flatten_parameters()
        self.decoder = nn.Linear(nhid, nout)
        self.softmax = nn.LogSoftmax(dim=-1)

        if pretrained:
            self.encoder.weight.data = vectors
            
    def init_weights(self):
        initrange = .1
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, reviews, labels, hidden):
        R = self.word_embedding(reviews)
        L = self.label_embedding(labels)
        L = torch.cat([L.unsqueeze(0)]*R.shape[0])
        X = torch.cat([R, L], -1)
        
        X, hidden = self.rnn(X, hidden)
        X = X.view(X.size(0)*X.size(1), X.size(2))
        
        X = self.decoder(X)
        log_probs = self.softmax(X)
        return log_probs, hidden
      

In [12]:
def repackage_hidden(h):
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)

In [22]:
def evaluate(data_source):
    # Turn on evaluation mode which disables dropout.
    model.eval()
    total_loss_e = 0
    total_number_of_words = 0

    with torch.no_grad():
        for i, batch in enumerate(data_source):
            labels = batch.theme.cuda().long() - 1
            batch = batch.review_text.cuda().long()
            hidden = None
            if batch.shape[0] > 1:
                data, targets = batch[:-1,:], batch[1:,:]
                number_of_words = data.shape[0]*data.shape[1]
                
                output, hidden = model(data, labels, hidden)
                output_flat = output.contiguous().view(-1, vocab_size)

                total_loss_e += criterion(output_flat, targets.contiguous().view(-1)).data.float()
                total_number_of_words += number_of_words
                hidden = repackage_hidden(hidden)
            
    return total_loss_e.item() / total_number_of_words

In [23]:
import numpy as np
import gc

In [24]:
def train(model, ep0, epN, train_iter, dev_iter, optimizer, criterion, 
          max_grad_norm, model_name, best_ppl = float('inf')):
    
    best_ppl = best_ppl
    
    len_train_iter = len(train_iter)
    for epoch in range(ep0, epN):
        model.train()
        total_loss_e = 0
        total_number_of_words = 0 
        
        for i, batch in enumerate(train_iter):

            labels = batch.theme.cuda().long() - 1
            batch = batch.review_text.cuda().long()
            hidden = None
            if batch.shape[0] > 1:
                data, targets = batch[:-1,:], batch[1:,:]
                number_of_words = data.shape[0]*data.shape[1]
                
                output, hidden = model(data, labels, hidden)
                hidden = repackage_hidden(hidden)

                output_flat = output.contiguous().view(-1, vocab_size)
                epoch_loss = criterion(output_flat, targets.contiguous().view(-1))
                total_loss_e += epoch_loss.data.float()
                total_number_of_words += number_of_words
            
                torch.nn.utils.clip_grad_norm_(model.parameters(),max_grad_norm)
                optimizer.zero_grad()
                epoch_loss.backward()
                optimizer.step()

                if i % 500 == 0:
                    cur_loss = epoch_loss.item() / number_of_words
                    tr_ppl_print = np.exp(cur_loss)
                    print("| epoch {:3d} | batch {} / {} | train_loss {} | train_ppl {}".format(
                            epoch, i, len_train_iter, 
                            np.round(cur_loss, 3), np.round(tr_ppl_print, 3)))

                #gc.collect()
                torch.cuda.empty_cache()
                
                if i % 5000 == 0:
                    cur_loss = total_loss_e.item() / total_number_of_words
                    tr_ppl_print = np.exp(cur_loss)
                    val_loss_eval = evaluate(dev_iter)
                    val_ppl_print = np.exp(val_loss_eval)
                    
                    template = "| epoch {:3d} | batch {} / {} | train_loss {} | train_ppl {} | val_loss {} | val_ppl {}"
                    print(template.format(
                            epoch, i, len_train_iter, 
                            np.round(cur_loss, 3), np.round(tr_ppl_print, 3), 
                            np.round(val_loss_eval, 3), np.round(val_ppl_print, 3)))

                    if val_ppl_print < best_ppl :
                        print('old best ppl {} new best ppl {}'.format(best_ppl, val_ppl_print))
                        best_ppl = val_ppl_print
                        best_model_name = 'best_model_{}_{}.model'.format(model_name, best_ppl)
                        print('save model...', best_model_name)
                        with open(best_model_name, 'wb') as file:
                            torch.save(model, file) 

                    gc.collect()
                    model.train()


In [25]:
with open('./best_model_base_model_ppl_99.14475134799753.model', 'rb') as file:
    model = torch.load(file)#BaseModel().cuda()
    model.eval()

In [26]:
# implement PPL
learning_rate = 0.001
criterion = nn.NLLLoss(reduction='sum', 
                       ignore_index=train_dataset.fields["review_text"].vocab.stoi['<pad>']).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
train(model,
      ep0 = 0,
      epN = 1,
      train_iter = train_iter,
      dev_iter = val_iter,
      optimizer = optimizer,
      criterion = criterion,
      max_grad_norm = 5,
      model_name = 'base_model_ppl',
      best_ppl = 99.14475134799753)

| epoch   0 | batch 500 / 21962 | train_loss 2.832 | train_ppl 16.985
| epoch   0 | batch 1000 / 21962 | train_loss 3.256 | train_ppl 25.955
| epoch   0 | batch 1500 / 21962 | train_loss 3.618 | train_ppl 37.259
| epoch   0 | batch 2000 / 21962 | train_loss 3.495 | train_ppl 32.935
| epoch   0 | batch 2500 / 21962 | train_loss 3.67 | train_ppl 39.243


| epoch   0 | batch 500 / 21962 | train_loss 2.932 | train_ppl 18.762
| epoch   0 | batch 1000 / 21962 | train_loss 3.303 | train_ppl 27.189
| epoch   0 | batch 1500 / 21962 | train_loss 3.431 | train_ppl 30.908
| epoch   0 | batch 2000 / 21962 | train_loss 3.313 | train_ppl 27.462
| epoch   0 | batch 2500 / 21962 | train_loss 3.475 | train_ppl 32.303
| epoch   0 | batch 3000 / 21962 | train_loss 3.954 | train_ppl 52.138
| epoch   0 | batch 3500 / 21962 | train_loss 3.463 | train_ppl 31.923
| epoch   0 | batch 4000 / 21962 | train_loss 3.518 | train_ppl 33.719
| epoch   0 | batch 4500 / 21962 | train_loss 3.505 | train_ppl 33.285
| epoch   0 | batch 5000 / 21962 | train_loss 3.228 | train_ppl 25.219
| epoch   0 | batch 5000 / 21962 | train_loss 3.487 | train_ppl 32.68 | val_loss 6.979 | val_ppl 1073.763
old best ppl inf new best ppl 1073.7626701856088
save model... best_model_base_model_ppl_1073.7626701856088.model


/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type BaseModel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


| epoch   0 | batch 5500 / 21962 | train_loss 4.429 | train_ppl 83.819
| epoch   0 | batch 6000 / 21962 | train_loss 4.274 | train_ppl 71.807
| epoch   0 | batch 6500 / 21962 | train_loss 4.038 | train_ppl 56.72
| epoch   0 | batch 7000 / 21962 | train_loss 4.28 | train_ppl 72.221
| epoch   0 | batch 7500 / 21962 | train_loss 3.804 | train_ppl 44.872
| epoch   0 | batch 8000 / 21962 | train_loss 4.425 | train_ppl 83.546
| epoch   0 | batch 8500 / 21962 | train_loss 4.624 | train_ppl 101.86
| epoch   0 | batch 9000 / 21962 | train_loss 4.54 | train_ppl 93.711
| epoch   0 | batch 9500 / 21962 | train_loss 3.895 | train_ppl 49.158
| epoch   0 | batch 10000 / 21962 | train_loss 4.277 | train_ppl 72.002
| epoch   0 | batch 10000 / 21962 | train_loss 4.066 | train_ppl 58.341 | val_loss 5.487 | val_ppl 241.568
old best ppl 1073.7626701856088 new best ppl 241.567588025592
save model... best_model_base_model_ppl_241.567588025592.model
| epoch   0 | batch 10500 / 21962 | train_loss 4.151 | train

RuntimeError: CUDA out of memory. Tried to allocate 4.49 GiB (GPU 0; 11.17 GiB total capacity; 7.44 GiB already allocated; 2.82 GiB free; 651.00 MiB cached)

In [ ]:
train(ep0 = 1,
      epN = 3,
      train_iter = train_iter,
      dev_iter = val_iter,
      optimizer = optimizer,
      criterion = criterion,
      max_grad_norm = 5,
      model_name = 'base_model_ppl',
      best_ppl = float('inf'))

In [ ]:
# Add <sos> and <eos>

In [59]:
vocab_size

30002

In [60]:
np.exp(evaluate(val_iter))

30083.068177308916